In [1]:
# import pandas_profiling as pp
import pandas as pd
import regex as re

In [2]:
df = pd.read_csv("/kaggle/input/thaijo-researcher-for-code-submission/test.csv")
df = df.fillna(" ")

In [3]:
def eng_(txt_):
    txt_l = txt_.lower()
    txt_c = txt_l.title()
    if len(txt_c.split())  > 4:
        return ""
    return txt_c
def thai_(txt_):
#   txt_ = txt_.lstrip("นาง")
    not_name = ['กองบรรณาธิการ วารสาร','กองบรรณาธิการ ผู้เชี่ยวชาญ','กองบรรณาธิการ กองบรรณาธิการ','สารบัญ วารสารวิทยาการจัดการ','สารบัญ สารบัญ']
    if len(txt_) > 30 or (txt_ in not_name):
        return ""
    return txt_

In [8]:
from itertools import chain
# '_source.author'
DEBUG = True
def soa(txt):
    # if DEBUG:
    #   print(txt.strip())
    prefixed = ['Assoc.Prof.Dr.' , 'ผศ.','ผศ.ดร.','รองศาสตราจารย์ ดร.', 'ร.ต.อ.', 'รศ.ดร.','ว่าที่ ร.ต.','นพ.' \
            'ผู้ช่วยศาสตราจารย์ ดร.','อ.ดร.','ดร.','นพ.','Mr.','Ven. Dr.','รศ.','กสทช.', 'Authors :',"Dr.","พระมหา","พระครู","Phramaha","Phra "\
               'Assoc. Prof' ,'Assist. Prof.Lect.', 'Asst.Prof.Dr.' ,'ว่าที่ พ.ต.','อาจารย์','ครู' ,"พระ" , "Phrakru","Phra" \
                ,"ผู้ช่วยศาสตราจารย์" ,"Assist. Prof." ,"Lect.", "Ms.","นาย","นาง","นางสาว","ม.ร.ว.","ว่าที่ร้อยตรี" ]
    txt_ = str(txt).strip()
    p_pattern = re.compile(r'\b(?:' + '|'.join(map(re.escape, prefixed)) + r')\s*')
    txt_ = p_pattern.sub("",txt_)
    txt_ = p_pattern.sub("",txt_)
    txt_ = re.sub('\d', '', txt_)



    pattern_eng = re.compile(r'^[A-Za-z]+ +[A-Za-z]+$')
    pattern_thai = re.compile(r'^[ก-๙]+ +[ก-๙]+$') #
    pattern = re.compile(r'([ก-๙\s]+)\s*\(\s*([A-Za-z\s]+)\s*\)')
    pattern_1 = re.compile(r'^[A-Za-z]+ +[A-Za-z]+ +\W+[ก-๙]+\W+')



    txt_ = txt_.strip("-*, (")


    ## ENG
    if pattern_eng.match(txt_):
        if txt_ == "et al":
            return ""

        txt_c = eng_(txt_)
        return txt_c


    ## THAI
    elif pattern_thai.match(txt_):
        txt_ = thai_(txt_)

        #print(txt_)
        return txt_

    ## THAI ENG
    elif pattern.match(txt_) or pattern_1.match(txt_):
        s = txt_
        txt_ = txt_.replace("(" ," ")
        a = txt_.split(" ")
        a = [i.strip('()') for i in a if i.strip()]
        thai = []
        eng = []
        for i in a:
            if re.findall(r'[A-Za-z]+',i):
                eng += [i]
            else:
                thai += [i]

        thai = thai_(" ".join(thai))
        eng = eng_(" ".join(eng))

        if pattern.match(s):
            return [thai,eng]

        if pattern_1.match(s):
            return [eng,thai]

    #others
    else:

        pattern_name1 = re.compile(r'^[ก-๙]+ +[ก-๙]+ +[A-Za-z]+ +[A-Za-z]+')
        pattern_name2 = re.compile(r'^[A-Za-z]+ +[A-Za-z]+ +[ก-๙]+ +[ก-๙]+')
        if pattern_name1.match(txt_) or pattern_name2.match(txt_):
            s1 = txt_
            thai = []
            eng = []
            b = txt_.split(" ")
            b = [i.strip() for i in b if i.strip()]
            if len(b) > 4:
                return ""
            for i in b:
                if re.findall(r'[A-Za-z]+',i):
                    eng += [i]
                else:
                    thai += [i]
            thai = thai_(" ".join(thai))
            eng = eng_(" ".join(eng))
            if pattern_name1.match(s1):
                return [thai,eng]

            if pattern_name2.match(s1):

                return [eng,thai]

        ## Others

        else:
            p_1 = re.compile(r'^[A-Za-zก-๙]+ +-+ +[[A-Za-zก-๙]+')
            if p_1.match(txt_):

                txt_ = [i.strip() for i in txt_.split('-')]
                txt_ = (" ".join(txt_)).title()
                return txt_

            if len(txt_) < 5:
                if txt_.strip() == '胡清国':
                    return "胡清国"
                return ""

            txt1 = [i.strip() for i in txt_.split(" ")]


            if len(txt1) <= 2:
               # print(txt1)
                not_word = ["ผู้ทรงคุณวุฒิ","แนะนำผู้เขียน",'บรรณาธิการ']
                for word in not_word:
                    if word in txt1:
                        txt1.remove(word)
                txt_ = " ".join(txt1)

                p_5 = re.compile("\D*[ก-ฮ]+\D*")
                p_6 = re.compile("\D*[A-Za-z]+\D*")
                if p_5.match(txt_):
                    # txt_ = [i.strip() for i in txt_.split()]

#############################################
# เหลือกรณี thai eng
                    if p_6.match(txt_):
                        res = []
                        txt_ = [i.strip('(')for i in txt_.split(" ")]
                        a = [(re.sub("[A-Za-z]","",i )).strip('@.') for i in txt_]
                        a = (" ".join(a)).strip()

                        return a
                    # delete ()
                    txt_ = [(re.sub("\W","",i)) for i in txt_.split(" ")]
                    txt_ = " ".join(txt_)
                    return txt_


                return txt_


            if len(txt1) == 3:
                if len(txt_) > 42:
                    return ""

                not_word = ["Cover Vol. ()"]
                if txt_ in not_word:
                    return ""

                #print(txt1)
                words = ["และคณะ","M.R.)","(ผู้แต่ง)"]
                for word in words:
                    if word in txt1:
                        txt1.remove(word)


                unseen = []
                for i in txt1:
                    if i not in unseen:
                        unseen.append(i)
                txt_ = " ".join(unseen)

                return txt_



            if len(txt1) >=4:
                words = ["และคณะ","and","other",'เกี่ยวกับวารสาร', 'ABOUT', 'THE', 'JOURNAL',"Assoc.", "Prof."]
                for word in words:
                    if word in txt1:
                        txt1.remove(word)
                txt1 = [i.strip(';[]()') for i in txt1 if i.strip()]
                if "ณ" in txt1:
                    return " ".join(txt1)

                if len(txt1)% 2 == 0:
                    pattern_name1 = re.compile(r'[ก-๙]+ [ก-๙]+ [A-Za-z]+ [A-Za-z]+')
                    pattern_name2 = re.compile(r'[A-Za-z]+ [A-Za-z]+ [ก-๙]+ [ก-๙]+')
                    txt_ = " ".join(txt1)
                    if pattern_name1.match(txt_) or pattern_name2.match(txt_):
                        thai = []
                        eng = []
                        b = txt_.split(" ")
                        b = [i.strip() for i in b if i.strip()]
                        for i in b:
                            if re.findall(r'[A-Za-z]+',i):
                                eng += [i]
                            else:
                                thai += [i]

                        thai = thai_(" ".join(thai))
                        eng = eng_(" ".join(eng))

                        return [thai,eng]

                    else:

                        p_3 = re.compile("[ก-๙]+ [ก-๙]+ [A-Za-z]+\D*")
                        p_4 = re.compile("\D+[ก-ฮ]+\D+")
                        if p_3.match(txt_) :
                            txt_ = txt_.split()
                            thai = thai_(" ".join(txt_[0:2]))
                            eng = eng_(" ".join(txt_[2:]))
                            return [thai,eng]


                        if p_4.match(txt_):
                            txt_ = [i.strip() for i in txt_.split()]
                            if len(txt_) == 4:
                                w1 = " ".join(txt_[0:2])
                                w2 = " ".join(txt_[2:4]).strip("และ")
                                return [w1,w2]

                            if len(txt_) == 6:
                                if re.match("[A-Za-z]+",txt_[0]):
                                    w1 = " ".join(txt_[0:3])
                                    w2 = " ".join(txt_[3:6])
                                    return [w1,w2]
                                else:
                                    w1 = " ".join(txt_[0:2])
                                    w2 = " ".join(txt_[2:4])
                                    w3 = " ".join(txt_[4:6])
                                    return [w1,w2,w3]
                        if len(txt_) > 36:

                            return ""
                        #######
                        # print(txt_)
                        return txt_
                else:
                    # print(txt1)
                    if len(txt1) > 7:
                        print(txt1)
                        return ""

                    if "Other" in txt1:
                        
                        txt1.remove("Other")

                    if len(txt1) ==4:
                        w1 = " ".join(txt1[0:2])
                        w2 = " ".join(txt1[2:])
                        return [w1,w2]

                    unseen = []
                    for i in txt1:
                        if i not in unseen:
                            unseen.append(i)
                    txt_ = " ".join(unseen)
                    return txt_

        # print(txt_)

In [9]:
#'_source.co-author'
DEBUG2 = False
def soca(txt):
  txt = str(txt)
  # if DEBUG2:
  #   print(txt)
  res = [i.strip() for i in txt.split(',')]
  res = [i.replace('"','') for i in res]
  res = [ soa(i) for i in res]



  return res

In [10]:
import pandas as pd
import regex as re


def remove_spaces(string):
    if string == '':
        return ''
    elif string == ' ':
        return ''
    if string[0] == ' ':
        string = string[1:]
    if string[-1] == ' ':
        string = string[:-1]
    return string


char = {'id': [], 'name': []}
df = pd.read_csv('/kaggle/input/thaijo-researcher-for-code-submission/test.csv')
df = df.fillna('')

for idx, row in df.iterrows():
    names = []
    char['id'].extend([f"{row['_id']}_{i+1}" for i in range(10)])
    author = soa(row['_source.author'])


    if author != '' and isinstance(author ,str):
        names.append(author)

    if isinstance(author,list):
        i = 0
        while i < len(author):
            names.append(author[i])
            i += 1

    # Manage co- author
    co_author = row['_source.co-author']
    if co_author != '':
        ca = soca(co_author)

        for c in ca:
            if isinstance(c,str):
                if c not in names:
                    names.append(c)
            if isinstance(c,list):
                i = 0
                while i < len(c):
                    if c[i] not in names:
                        names.append(c[i])
                    i += 1



    if len(names) == 0:
        char['name'].extend([None]*10)
        continue

    names = [remove_spaces(name) for name in names]
    if len(names) > 10:
        names = names[:10]
    elif len(names) < 10:
        names.extend(['']*(10-len(names)))
    char['name'].extend(names)

char = pd.DataFrame(char)
char.to_csv('f1.csv', index=False)

['subcommittee', 'on', 'the', 'preparation', 'of', 'Monographs', 'of', 'Selected', 'Thai', 'Materia', 'Medica']
['subcommittee', 'on', 'the', 'preparation', 'of', 'Monographs', 'of', 'Selected', 'Thai', 'Materia', 'Medica']
['subcommittee', 'on', 'the', 'preparation', 'of', 'monofraphs', 'of', 'selected', 'thai', 'materia', 'medica']
['subcommittee', 'on', 'the', 'preparation', 'of', 'monofraphs', 'of', 'selected', 'thai', 'materia', 'medica']
